In [1]:

import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import tensorflow as tf
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.layers import GlobalAveragePooling2D

2025-07-28 16:17:09.283473: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-28 16:17:09.583668: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-28 16:17:10.489089: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [36]:
# EfficientNetV2B0 - 22.58 MB
# NASNetMobile - 16.29 MB
# EfficientNetB0 - 15 MB
# MobileNetV2 - 8.61 MB
# MobileViT - 4.9 MB - Their findings suggest that, performance-wise, MobileViT is better than other models with the same or higher complexity (MobileNetV3, for example), while being efficient on mobile devices.
# MobileNetV3Small - 3.58 MB
from mobilevit import create_mobilevit


# pretrained_model = tf.keras.applications.MobileNetV3Smalls(
#                     input_shape=(224,224, 3),
#                     include_top=False,
#                     weights='imagenet',
#                     pooling='avg'
#                     )
# pretrained_model.trainable = False
# print(pretrained_model.summary())

pretrained_model = create_mobilevit(256)
pretrained_model.trainable = False
# print(pretrained_model.summary())

In [ ]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(64, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(32, activation='relu')(x)

# example of multi-output model
gender = tf.keras.layers.Dense(5, activation="softmax", name='gender')(x)
master_category = tf.keras.layers.Dense(5,  activation="softmax", name='master_category')(x)
sub_category = tf.keras.layers.Dense(31, activation="softmax", name='sub_category')(x)
arti_type = tf.keras.layers.Dense(76,  activation="softmax", name='arti_type')(x)
base_color = tf.keras.layers.Dense(46, activation="softmax", name='base_color')(x)
season = tf.keras.layers.Dense(4,  activation="softmax", name='season')(x)
usage = tf.keras.layers.Dense(8, activation="softmax", name='usage')(x)

outputs = [gender , master_category , sub_category , arti_type , base_color , season , usage]
model = tf.keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_17        │ (None, 256, 256,  │          0 │ input_layer_20[0… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_106 (Conv2D) │ (None, 128, 128,  │        448 │ rescaling_17[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_107 (Conv2D) │ (None, 128, 128,  │        512 │ conv2d_106[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_107[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ silu_56 (Silu)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_28 │ (None, 128, 128,  │        288 │ silu_56[0][0]     │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ silu_57 (Silu)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_108 (Conv2D) │ (None, 128, 128,  │        512 │ silu_57[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         64 │ conv2d_108[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_98 (Add)        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │ conv2d_106[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_109 (Conv2D) │ (None, 128, 128,  │        512 │ add_98[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_109[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ silu_58 (Silu)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_24   │ (None, 129, 129,  │          0 │ silu_58[0][0]     │
│ (ZeroPadding2D)     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_29 │ (None, 64, 64,    │        288 │ zero_padding2d_2

 Total params: 1,334,415 (5.09 MB)

 Trainable params: 28,399 (110.93 KB)

 Non-trainable params: 1,306,016 (4.98 MB)

None


In [ ]:
# datasets i have:
# cloth_dataset - images in separate folders
# clothing dataset-full - images in one folder with a .csv file (image,sender_id,label,kids, where label is a category: skirt, pants, etc.)
# fashion-clothes - images in separate folders (only coats, hoodies, men suits, dresses)
# fashionMNIST - images in separate folders (0. T-shirt/top, 1. Trouser, 2. Pullover, 3. Dress, 4. Coat, 5. Sandal, 6. Shirt, 7. Sneaker, 8. Bag, 9. Ankle boot)
# new-data-fashio - images in separate folders (black or blue dress, pants, shirt, shoes and shorts)

In [10]:
import kagglehub
# downloaded
# path = kagglehub.dataset_download("dqmonn/zalando-store-crawl") #zalando clothes - folders
# kagglehub.dataset_download("hammaadali/real-fashion") #in folders
# kagglehub.dataset_download("mayukh18/fashion200k-dataset") #in folders, there are also some labels
# kagglehub.dataset_download("kritanjalijain/outfititems") #in folders
# kagglehub.dataset_download("sivaprasads/fashion-dataset") #- only images without folders or file
# kagglehub.dataset_download("anchalsekhrii/fashion-classification-dataset") #in folders
# kagglehub.dataset_download("aloofzebra03/clothes-classification-final-aug-split") #in folders
path = kagglehub.dataset_download("kaborg15/vibrent-clothes-rental-dataset") #- maybe????  CC BY 4.0 - csv file very difficult to process, has labrls in an array


# kagglehub.dataset_download("tuannguyenvananh/fashion-crawled-images")# in folders, has culottes is big 81 gb

100%|██████████| 6.17G/6.17G [10:25<00:00, 10.6MB/s]  

Extracting files...
